# Live Plotting Cookbook

Real-time visualization of sweep data as it's collected.

**⚠️ IMPORTANT: Streaming sweeps support 1D data only.** For 2D/ND sweeps, collect data first, then use `log_sweep()`.

## Two APIs for Live Plotting

1. **`sweep()` context manager** - NEW! For streaming 1D data incrementally
   - Use when: Collecting data points one-by-one in real-time
   - Benefit: One sweep record in file, live updates during collection
   - Limitation: 1D only (scalar x, scalar y)

2. **`log_sweep()` method** - Traditional approach for complete sweeps
   - Use when: You have all data at once, or need 2D/ND sweeps
   - Benefit: Works with any dimensionality
   - Note: Multiple calls create multiple sweep records

## Two Backends

- **server**: Plots in browser window (works everywhere)
- **inline**: Plots in notebook cells (this notebook)

## Setup

In [ ]:
import tempfile
import time
import numpy as np
from stanza.logger.data_logger import DataLogger
from stanza.plotter import enable_live_plotting

## Example 1: Basic Inline Plotting

Simplest case - one sweep, inline display.

In [ ]:
# Create logger and enable inline plotting
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="basic", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")

In [ ]:
# Create session and log data
session = logger.create_session()

# First batch - plot appears
x = np.linspace(0, 5, 10)
y = np.sin(x)
session.log_sweep(name="signal", x_data=x, y_data=y, x_label="Time (s)", y_label="Amplitude")
session.flush()
print(f"Logged {len(x)} points")

In [ ]:
# Second batch - same plot updates
x = np.linspace(5, 10, 10)
y = np.sin(x)
session.log_sweep(name="signal", x_data=x, y_data=y, x_label="Time (s)", y_label="Amplitude")
session.flush()
print(f"Total: {len(session._buffer) if hasattr(session, '_buffer') else 'N/A'} points")

In [ ]:
# Cleanup
session.close()

## Example 2: Streaming Data with Context Manager

Use `sweep()` context for real-time streaming. Data accumulates and writes once to file on exit.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="streaming", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Stream data in chunks using context manager
with session.sweep("decay", x_label="Time (s)", y_label="Signal") as s:
    for i in range(20):
        t = i * 0.5
        amplitude = np.cos(2 * np.pi * 0.5 * t) * np.exp(-t/5)
        
        # append() streams to live plot immediately, accumulates for file
        s.append([t], [amplitude])
        time.sleep(0.1)  # Simulate measurement time

# Sweep written to file here (exactly one record)
print("✓ Streaming complete - one sweep record written")

In [ ]:
session.close()

## Example 3: Multiple Streaming Plots

Different sweep names create separate plots. Use nested contexts or sequential contexts.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="multi", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Create two streaming sweeps for I and Q components
t = np.linspace(0, 10, 50)

# Start both sweeps
s_I = session.start_sweep("I", x_label="Time", y_label="I")
s_Q = session.start_sweep("Q", x_label="Time", y_label="Q")

for time_point in t:
    I = np.cos(2 * np.pi * time_point)
    Q = np.sin(2 * np.pi * time_point)
    
    # Append to both sweeps - both plots update in real-time
    s_I.append([time_point], [I])
    s_Q.append([time_point], [Q])
    time.sleep(0.05)

# Complete both sweeps
s_I.end()
s_Q.end()

print("✓ Two plots created, two sweep records written")

In [ ]:
session.close()

## Example 4: Server Backend (Browser)

Use this for longer experiments or when not in a notebook.

In [ ]:
# Setup with server backend
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="server_demo", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="server", port=5007)

print("\n⚠️  Open http://localhost:5006 in your browser NOW\n")

In [ ]:
# Wait a moment for browser to connect
time.sleep(2)

# Now stream data - it appears in browser
session = logger.create_session()

with session.sweep("noisy_signal", x_label="Time", y_label="Signal + Noise") as s:
    for i in range(30):
        x = i * 0.2
        y = np.sin(x) + 0.1 * np.random.randn()
        s.append([x], [y])
        time.sleep(0.1)

print("✓ Check browser for plot")

In [ ]:
session.close()
backend.stop()

## Example 5: Rabi Oscillations

Realistic quantum experiment simulation.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="rabi", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Sweep pulse duration with streaming context
pulse_times = np.linspace(0, 20, 40)  # microseconds

with session.sweep(
    "rabi_oscillations",
    x_label="Pulse Duration (μs)",
    y_label="Excited State Population"
) as s:
    for t_pulse in pulse_times:
        # Simulate Rabi oscillation with decay
        omega_rabi = 2 * np.pi * 0.5  # MHz
        T1 = 30  # microseconds
        
        population = 0.5 * (1 - np.cos(omega_rabi * t_pulse)) * np.exp(-t_pulse / T1)
        
        # Add measurement noise
        population += 0.02 * np.random.randn()
        
        s.append([t_pulse], [population])
        time.sleep(0.05)

print("✓ Rabi sweep complete - one sweep record written")

In [ ]:
session.close()

## Key Points

### Using `sweep()` Context Manager (Recommended for Streaming)
1. **No flush() needed** - `append()` updates plot immediately, context exit writes to file
2. **One file record** - Complete sweep written once on context exit
3. **1D only** - Both x and y must be scalar values per append
4. **Same name = same plot** - Different names create separate plots
5. **Exception = no file write** - Live updates shown, but sweep not persisted if error occurs
6. **Use cancel()** - Explicitly skip file write: `s.cancel()`

### Using `log_sweep()` Method (Traditional)
1. **Call `flush()`** - Only flushed data appears in plots
2. **Multiple calls = multiple records** - Each `log_sweep()` creates a new sweep record
3. **Any dimensionality** - Works with 1D, 2D, or ND sweeps
4. **Same name = same plot** - Different names create separate plots

### Backends
- **Inline backend** - Requires `jupyter_bokeh` package
- **Server backend** - Open browser to `http://localhost:PORT` before logging
- **Updates are automatic** - Just append/flush and watch

## When to Use Which API

**Use `sweep()` context manager when:**
- Collecting data incrementally in real-time (one point at a time)
- Want live plotting with single file record
- Data is 1D (scalar x, scalar y)
- Example: Time-series measurements, parameter sweeps

**Use `log_sweep()` when:**
- You have complete data already collected
- Need 2D/ND sweeps (e.g., heatmaps)
- Want explicit control over each write
- Example: Post-processing, batch uploads

## Troubleshooting

**Plot doesn't appear:**
- For `sweep()`: Is bokeh backend enabled? Did context enter?
- For `log_sweep()`: Did you call `session.flush()`?
- For inline: Is `jupyter_bokeh` installed?
- For server: Is browser open to the right URL?

**Plot doesn't update:**
- For `sweep()`: Are you calling `append()`? Is sweep still active?
- For `log_sweep()`: Using same sweep name? Did you flush?

**ValueError: Only 1D sweeps supported:**
- You're using 2D/ND data with `sweep()` context
- Solution: Use `log_sweep()` instead after collecting complete data

**Sweep already active error:**
- Can't start two sweeps with same name simultaneously
- Solution: Call `end()` on first sweep, or use different name

**Address already in use:**
- Another server on that port
- Use different port: `enable_live_plotting(logger, backend="server", port=5007)`

## Advanced: Explicit Lifecycle

Instead of context manager, you can manually control sweep lifecycle:

```python
s = session.start_sweep("my_sweep", x_label="X", y_label="Y")
for x, y in data_stream:
    s.append([x], [y])
s.end()  # or s.cancel() to skip file write
```